In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
import os
import random 

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# read in original data
train_data = "hida_workspace/trainSet/trainSet.txt"
test_data = "hida_workspace/testSet/testSet.txt"

df_train = pd.read_csv(train_data)
df_train_length_idx = len(df_train)
df_test = pd.read_csv(test_data)

# Merge the two datasets
train_test = [df_train, df_test]
df_train_test = pd.concat(train_test)
# df_train_test['Prognosis'].loc[df_train_test['Prognosis'] == '<undefined>'] = np.nan
df_train_test.loc[df_train_test['Prognosis'] == '<undefined>', 'Prognosis'] = np.nan

# all variables in the dataset (incl. outcome)
variables = list(df_train_test.columns[3:])


# read in csv from Tobias (imputed via catboost
df_imputed_train = pd.read_csv("Results/Tobias/imputed_train.csv")
df_imputed_test = pd.read_csv("Results/Tobias/imputed_test.csv")


df_for_xg = df_train.copy()
df_for_xg["Prognosis"] = df_for_xg["Prognosis"].astype("category").cat.codes
df_for_xg = df_for_xg[variables]


In [22]:
# get predictions from xgboost not imputed:
xgboost_features = variables.copy()
xgboost_features.remove("Prognosis")

train_valid_split = 0.9
sep_index = int(863 * train_valid_split)

X_train = df_for_xg[xgboost_features].iloc[0:sep_index]
y_train = df_for_xg["Prognosis"].iloc[0:sep_index]


X_test = df_for_xg[xgboost_features].iloc[sep_index:]
y_test = df_for_xg["Prognosis"].iloc[sep_index:]
image_names = df_train["ImageFile"].iloc[sep_index:]

# fit model on training data
model = XGBClassifier()
model.fit(X_train.values, y_train.values)


# make predictions for test data
y_pred = model.predict(X_test.values)
xgboost_results = dict(predictions_xg=list(y_pred), images=image_names.to_list(), targets=y_test.values)
xgboost_results_df = pd.DataFrame(xgboost_results)

[12:02:07] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/haicore-project-covchl/do8557/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/haicore-project-covchl/do8557/.local/lib/python3.6/site-packages/xgboost/data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


In [23]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [24]:
# get predictions from xgboost imputed:
xgboost_features = variables.copy()
xgboost_features.remove("Prognosis")


train_valid_split = 0.9
sep_index = int(863 * train_valid_split)

X_train = df_imputed_train[xgboost_features].iloc[0:sep_index]
y_train = df_imputed_train["Prognosis"].iloc[0:sep_index]

X_test = df_imputed_train[xgboost_features].iloc[sep_index:]
y_test = df_imputed_train["Prognosis"].iloc[sep_index:]
image_names = df_train["ImageFile"].iloc[sep_index:]





# fit model on training data
model = XGBClassifier()
model.fit(X_train.values, y_train.values)


# make predictions for test data
y_pred = model.predict(X_test.values)
xgboost_imputed_results = dict(predictions_xg_imputed=list(y_pred), images=image_names.to_list())
xgboost_imputed_results_df = pd.DataFrame(xgboost_imputed_results)

[12:02:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [25]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [26]:
xgboost_imputed_results_df

,predictions_xg_imputed,images
0,1.0,P_678.png
1,1.0,P_673.png
2,0.0,P_669.png
3,1.0,P_675.png
4,1.0,P_640.png
...,...,...
82,1.0,P_1_12.png
83,1.0,P_1_8.png
84,0.0,P_1_10.png
85,1.0,P_1_26.png


In [27]:
xgboost_results_df

,predictions_xg,images,targets
0,1,P_678.png,1
1,0,P_673.png,1
2,0,P_669.png,0
3,1,P_675.png,1
4,1,P_640.png,1
...,...,...,...
82,1,P_1_12.png,1
83,1,P_1_8.png,1
84,0,P_1_10.png,0
85,1,P_1_26.png,1


In [28]:
# stack predicions:
xgboost_imputed_results_df = xgboost_imputed_results_df.set_index("images")
xgboost_results_df = xgboost_results_df.set_index("images")

all_predictions = pd.concat([xgboost_imputed_results_df, xgboost_results_df], axis=1)

In [29]:
all_predictions

,predictions_xg_imputed,predictions_xg,targets
images,,,
P_678.png,1.0,1,1
P_673.png,1.0,0,1
P_669.png,0.0,0,0
P_675.png,1.0,1,1
P_640.png,1.0,1,1
...,...,...,...
P_1_12.png,1.0,1,1
P_1_8.png,1.0,1,1
P_1_10.png,0.0,0,0


In [30]:
(all_predictions['predictions_xg_imputed']==all_predictions['predictions_xg']).sum()

71

In [31]:
(all_predictions['predictions_xg_imputed']== all_predictions['targets']).sum()

65

In [32]:
(all_predictions['predictions_xg']== all_predictions['targets']).sum()

69